### Solves the 1D Inviscid Burgers' equation 
Initial Condition: $e^{-x^2/50}$

In [1]:
import numpy as np
from core import *

In [2]:
import matplotlib.pyplot as plt
from matplotlib import animation, rcParams
from IPython.display import HTML

In [3]:
rcParams['animation.html'] = 'html5'

In [4]:
# Domain's boundaries
west = -15.
east = 15.

# Operator's order of accuracy
k = 2

# Number of grid cells
m = 300
dx = (east - west) / m

# Simulation time
t = 5

# CFL condition for explicit schemes
dt = dx

# 1D Mimetic divergence operator
D = div1D(k, m, dx)
# 1D 2nd order interpolator
I = interpol1D(m, 0.5)

# 1D  Staggered grid
xgrid = np.append(np.insert(np.arange(west+dx/2, east, dx), 0, west), east)

# Impose IC
U = np.exp(-(xgrid**2) / 50.)

# Premultiply out of the time loop (since it doesn't change)
D = -dt * np.dot(D, I)

/Users/nutrik/Documents/Science/scodes/Python/pymole/core/div1D.py:53: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  D[i, j:j+k] = coeffs
/Users/nutrik/Documents/Science/scodes/Python/pymole/core/interpol1D.py:27: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  I[0, 0] = 1.
/Users/nutrik/Documents/Science/scodes/Python/pymole/core/interpol1D.py:28: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  I[-1, -1] = 1.
/Users/nutrik/Documents/Science/scodes/Python/pymole/core/interpol1D.py:37: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  I[i, j:j+2] = avg


In [5]:
def animate_burgers(solution):
    fig = plt.figure(figsize=(8,6))
    ax = plt.gca()
    ax.set_xlim((-15, 15))
    ax.set_ylim((0.1, 1.1))
    ax.set_xlabel('x')
    ax.set_ylabel('u(x)')
    plt.grid(True)
    line, = ax.plot([], [], lw=2)
    plt.close()

    def animate(i):
        line.set_data(xgrid, solution[:, i])
        ax.set_title('t = {:01.2f}s, Mass: {:03.4f}'.format(i * dt, np.trapz(solution[:, i])))
        return line,
    
    return animation.FuncAnimation(fig, animate, frames=solution.shape[1], interval=50, blit=True)

In [6]:
Nt = int(t/dt)
solution = np.zeros((U.shape[0], Nt+1))
solution[:, 0] = U

# Time integration loop
for i in range(Nt):
    # Euler's scheme
    U2 = U + U * (D @ U)
    U = U2.copy()
    solution[:, i+1] = U.copy()

In [7]:
anim=animate_burgers(solution)

In [8]:
anim